In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from PIL import Image
import numpy as np
import glob

data_bottle_train = glob.glob('/content/drive/MyDrive/mvtec_anomaly_detection/bottle/train /*')
data_bottle_test = {
    'broken_large': glob.glob('/content/drive/MyDrive/mvtec_anomaly_detection/bottle/test/broken_large/*.png'),
    'broken_small': glob.glob('/content/drive/MyDrive/mvtec_anomaly_detection/bottle/test/broken_small/*.png'),
    'contamination': glob.glob('/content/drive/MyDrive/mvtec_anomaly_detection/bottle/test/contamination/*.png'),
    'good': glob.glob('/content/drive/MyDrive/mvtec_anomaly_detection/bottle/test/good/*.png')
}

In [ ]:
x = []
y = []

# train 데이터 처리
for file_path in data_bottle_train:
    img = Image.open(file_path)
    img = img.resize((224, 224))
    img = img.convert("RGB")
    img = np.array(img) / 255.0
    x.append(img)
    y.append(0)  # 정상

# test 데이터 처리
for category, files in data_bottle_test.items():
    is_defective = (category != 'good')  # 불량 이미지인 경우 True, 정상 이미지인 경우 False
    for file_path in files:
        img = Image.open(file_path)
        img = img.resize((224, 224))
        img = img.convert("RGB")
        img = np.array(img) / 255.0
        x.append(img)
        y.append(1 if is_defective else 0)  # 불량인 경우 1, 정상인 경우 0

x = np.array(x)
y = np.array(y)

In [ ]:
#학습 모델
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

# 데이터 분할
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# 모델 구성
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # 불량 여부를 이진 분류로 출력 (0 또는 1)
])

# 모델 컴파일
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 모델 학습
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

# 모델 평가
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)

Epoch 1/10
3/3 [==============================] - 2s 142ms/step - loss: 3.7373 - accuracy: 0.5000 - val_loss: 0.7378 - val_accuracy: 0.8235
Epoch 2/10
3/3 [==============================] - 0s 58ms/step - loss: 0.8012 - accuracy: 0.7424 - val_loss: 0.7544 - val_accuracy: 0.1765
Epoch 3/10
3/3 [==============================] - 0s 53ms/step - loss: 0.6614 - accuracy: 0.5909 - val_loss: 0.5587 - val_accuracy: 0.8235
Epoch 4/10
3/3 [==============================] - 0s 49ms/step - loss: 0.7548 - accuracy: 0.7424 - val_loss: 0.5027 - val_accuracy: 0.8235
Epoch 5/10
3/3 [==============================] - 0s 49ms/step - loss: 0.6183 - accuracy: 0.7424 - val_loss: 0.6590 - val_accuracy: 0.8235
Epoch 6/10
3/3 [==============================] - 0s 48ms/step - loss: 0.6608 - accuracy: 0.7424 - val_loss: 0.5515 - val_accuracy: 0.8235
Epoch 7/10
3/3 [==============================] - 0s 48ms/step - loss: 0.6070 - accuracy: 0.7424 - val_loss: 0.4896 - val_accuracy: 0.8235
Epoch 8/10
3/3 [==========